In [2]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
from collections import Counter
import itertools
import seaborn as sns
from subprocess import check_output
%matplotlib inline

In [2]:
train = pd.read_csv("D:/Users/Ziqiu/OneDrive/Documents/University Docs/Stat441/DataChallenge 1/fashion-mnist_train.csv")
#print(x_train.shape)
#x_train.head()
test = pd.read_csv("D:/Users/Ziqiu/OneDrive/Documents/University Docs/Stat441/DataChallenge 1/test_data.csv")
test.shape

(10000, 785)

In [3]:
z_train = Counter(train['label'])
z_train

Counter({2: 6000,
         9: 6000,
         6: 6000,
         0: 6000,
         3: 6000,
         4: 6000,
         5: 6000,
         8: 6000,
         7: 6000,
         1: 6000})

In [4]:
x_train = train.ix[:,1:].values.astype('float32') # all pixel values
y_train = train.ix[:,0].values.astype('int32') # only labels i.e targets digits
x_test = test.ix[:,1:].values.astype('float32')

C:\Users\Ziqiu Zhu\Documents\Anaconda\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  """Entry point for launching an IPython kernel.
C:\Users\Ziqiu Zhu\Documents\Anaconda\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  
C:\Users\Ziqiu Zhu\Documents\Anaconda\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  This is separate from th

In [6]:
from sklearn.preprocessing import Normalizer

scaler_xtrain = Normalizer().fit(x_train)
normalized_xtrain = scaler_xtrain.transform(x_train)

scaler_xtest = Normalizer().fit(x_test)
normalized_xtest = scaler_xtest.transform(x_test)

In [8]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from keras.layers.normalization import BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau
from sklearn.model_selection import train_test_split
batch_size = 32
num_classes = 10
epochs = 60
input_shape = (28, 28, 1)

Using TensorFlow backend.


In [9]:
X_train = x_train.reshape(x_train.shape[0], 28, 28,1)
X_test = x_test.reshape(x_test.shape[0], 28, 28,1)
y_train

array([2, 9, 6, ..., 8, 8, 7])

In [10]:
y_train = keras.utils.to_categorical(y_train, num_classes)
X_train, X_val, Y_train, Y_val = train_test_split(X_train, y_train, test_size = 0.07, random_state=42)

In [11]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),activation='relu',kernel_initializer='he_normal',input_shape=input_shape))
model.add(Conv2D(32, kernel_size=(3, 3),activation='relu',kernel_initializer='he_normal'))
model.add(MaxPool2D((2, 2)))
model.add(Dropout(0.15))
model.add(Conv2D(64, (3, 3), activation='relu',padding='same',kernel_initializer='he_normal'))
model.add(Conv2D(64, (3, 3), activation='relu',padding='same',kernel_initializer='he_normal'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.20))
model.add(Conv2D(64, (3, 3), activation='relu',padding='same',kernel_initializer='he_normal'))
model.add(Conv2D(64, (3, 3), activation='relu',padding='same',kernel_initializer='he_normal'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(128, (3, 3), activation='relu',padding='same',kernel_initializer='he_normal'))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.25))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.RMSprop(),
              metrics=['accuracy'])

learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc', 
                                            patience=3, 
                                            verbose=1, 
                                            factor=0.5, 
                                            min_lr=0.0001)

datagen = ImageDataGenerator(
        featurewise_center=False,
        samplewise_center=False, 
        featurewise_std_normalization=False, 
        samplewise_std_normalization=False, 
        zca_whitening=False,  
        rotation_range=15, 
        zoom_range = 0.1, 
        width_shift_range=0.1,  
        height_shift_range=0.1,  
        horizontal_flip=False,  
        vertical_flip=False) 

In [12]:
y_train.shape

(60000, 10)

In [13]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 24, 24, 32)        9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 12, 12, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 12, 12, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 12, 12, 64)        18496     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 12, 12, 64)        36928     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 6, 6, 64)          0         
__________

In [14]:
datagen.fit(X_train)
h = model.fit_generator(datagen.flow(X_train,Y_train, batch_size=batch_size),
                              epochs = epochs, validation_data = (X_val,Y_val),
                              verbose = 1, steps_per_epoch=X_train.shape[0] // batch_size
                              , callbacks=[learning_rate_reduction],)

Epoch 1/60
1743/1743 [==============================] - 28s 16ms/step - loss: 0.8323 - acc: 0.6905 - val_loss: 0.4636 - val_acc: 0.8136
Epoch 2/60
1743/1743 [==============================] - 24s 14ms/step - loss: 0.5355 - acc: 0.8035 - val_loss: 0.3748 - val_acc: 0.8569
Epoch 3/60
1743/1743 [==============================] - 25s 14ms/step - loss: 0.4528 - acc: 0.8375 - val_loss: 0.2928 - val_acc: 0.8888
Epoch 4/60
1743/1743 [==============================] - 24s 14ms/step - loss: 0.4205 - acc: 0.8482 - val_loss: 0.2882 - val_acc: 0.8933
Epoch 5/60
1743/1743 [==============================] - 25s 15ms/step - loss: 0.3972 - acc: 0.8582 - val_loss: 0.3250 - val_acc: 0.8852
Epoch 6/60
1743/1743 [==============================] - 27s 15ms/step - loss: 0.3802 - acc: 0.8643 - val_loss: 0.3012 - val_acc: 0.8945
Epoch 7/60
1743/1743 [==============================] - 27s 15ms/step - loss: 0.3710 - acc: 0.8675 - val_loss: 0.3011 - val_acc: 0.9007
Epoch 8/60
1743/1743 [==========================

In [15]:
final_loss, final_acc = model.evaluate(X_val, Y_val, verbose=0)
print("Final loss: {0:.6f}, final accuracy: {1:.6f}".format(final_loss, final_acc))

Final loss: 0.198443, final accuracy: 0.930000


In [17]:
#get the predictions for the test data
predicted_classes = model.predict_classes(X_test)

#get the indices to be plotted
y_true = test.iloc[:, 0]
correct = np.nonzero(predicted_classes==y_true)[0]
incorrect = np.nonzero(predicted_classes!=y_true)[0]

In [18]:
predicted_classes.shape

(10000,)

In [3]:
submissions=pd.DataFrame({"ids": list(range(0,len(predicted_classes))),
                         "label": predicted_classes})
submissions.to_csv("Submit3.csv", index=False, header=True)

NameError: name 'predicted_classes' is not defined